# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure gmaps

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_cities = pd.read_csv("../Weatherpy/Final Outputs/weathercities.csv")
weather_cities.dropna()

,Unnamed: 0,City,Lat,Long,Date,Cloudiness,Humidity,Max Temp,Min Temp,Windspeed,Lng
0,0,hambantota,6.1241,,1627344650,97,82,79.90,79.90,16.91,81.1185
1,1,vaini,-21.2000,,1627344631,75,83,78.96,78.96,10.36,-175.2000
2,2,aswan,24.0934,,1627344651,20,20,86.77,86.77,11.81,32.9070
3,3,nikolskoye,59.7035,,1627344651,10,71,62.02,60.40,3.56,30.7861
4,5,victoria,22.2855,,1627344122,95,84,89.33,86.95,5.99,114.1577
...,...,...,...,...,...,...,...,...,...,...,...
522,574,klyuchi,52.2667,,1627344788,0,75,57.54,57.54,5.68,79.1667
523,575,murgab,37.4966,,1627344788,0,25,87.24,87.24,14.47,61.9714
524,576,tuy hoa,13.0833,,1627344788,31,63,84.88,84.88,17.31,109.3000
525,577,severo-kurilsk,50.6789,,1627344388,100,87,60.76,60.76,7.43,156.1250


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Lat and Lng as locations
locations = weather_cities[["Lat","Lng"]].astype(float)

#Humidty as weight
humidity = weather_cities["Humidity"]

In [8]:
#gmap config

gmaps.configure(api_key=g_key)

# Create Heatmap
fig = gmaps.figure()

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 4)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
great_weather_cities = weather_cities.loc[(weather_cities["Windspeed"] <= 20) & (weather_cities["Humidity"]<= 40) & \
                  (weather_cities["Max Temp"] >= 70) & (weather_cities["Max Temp"] <= 90)].dropna()
great_weather_cities.count()

Unnamed: 0    21
City          21
Lat           21
Long          21
Date          21
Cloudiness    21
Humidity      21
Max Temp      21
Min Temp      21
Windspeed     21
Lng           21
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Hotel Dataframe

hotel_df = great_weather_cities.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

hotel_df

#parameters for hotels with 5000 meters

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
       
    print("-----Completed-------")

Retrieving Results for Index 0: aswan.
Closest hotel in aswan is Sofitel Legend Old Cataract Aswan.
-----Completed-------
Retrieving Results for Index 1: la ronge.
Closest hotel in la ronge is The Harbour Inn.
-----Completed-------
Retrieving Results for Index 2: port elizabeth.
Closest hotel in port elizabeth is Island Way Villa.
-----Completed-------
Retrieving Results for Index 3: thompson.
Closest hotel in thompson is Best Western Thompson Hotel & Suites.
-----Completed-------
Retrieving Results for Index 4: komsomolskiy.
Closest hotel in komsomolskiy is TANTANA HOTEL FERGANA.
-----Completed-------
Retrieving Results for Index 5: emba.
Could not find result
-------------------------
-----Completed-------
Retrieving Results for Index 6: flin flon.
Closest hotel in flin flon is Victoria Inn Flin Flon.
-----Completed-------
Retrieving Results for Index 7: tabas.
Closest hotel in tabas is Bahman Hotel.
-----Completed-------
Retrieving Results for Index 8: zabol.
Could not find result
-

In [11]:
#Display the results
hotel_df

,Unnamed: 0,City,Lat,Long,Date,Cloudiness,Humidity,Max Temp,Min Temp,Windspeed,Lng,Hotel Name
0,2,aswan,24.0934,,1627344651,20,20,86.77,86.77,11.81,32.9070,Sofitel Legend Old Cataract Aswan
1,39,la ronge,55.1001,,1627344658,26,39,70.56,70.56,2.73,-105.2842,The Harbour Inn
2,58,port elizabeth,-33.9180,,1627344302,80,33,70.11,70.11,9.75,25.5701,Island Way Villa
3,78,thompson,55.7435,,1627344666,4,22,76.08,76.08,6.26,-97.8558,Best Western Thompson Hotel & Suites
4,97,komsomolskiy,40.4272,,1627344671,3,27,81.82,72.75,4.72,71.7189,TANTANA HOTEL FERGANA
5,110,emba,48.8264,,1627344674,90,40,73.65,73.65,6.17,58.1442,
6,144,flin flon,54.7682,,1627344426,66,29,77.63,77.63,11.23,-101.8650,Victoria Inn Flin Flon
7,156,tabas,33.5959,,1627344522,0,24,89.64,89.64,15.57,56.9244,Bahman Hotel
8,179,zabol,32.2500,,1627344689,100,16,77.25,77.25,3.02,67.2500,
9,262,irricana,51.3237,,1627344707,0,26,85.96,80.31,9.86,-113.6048,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure

fig


Figure(layout=FigureLayout(height='420px'))